In [1]:
import mediapipe as mp
import numpy as np
import cv2
from sklearn.metrics import accuracy_score 
import pickle 
import datetime
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

In [7]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [8]:
model

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression(max_iter=2100))]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier(max_iter=2100))]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [5]:
df=pd.read_csv('coords.csv',sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [6]:
X = df.drop('class', axis=1) 
y = df['class'] 

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

In [8]:
for algo, model in model.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 0.9959839357429718
rc 0.9939759036144579
rf 0.9959839357429718
gb 0.9959839357429718


In [9]:
ans=[]
count=0
flag=True
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened() and flag:
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                                 mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
            # Concate rows
            row = pose_row+face_row
            
            
            # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row)

            #make detection
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            if len(ans)==0 or ans[-1][1]!=body_language_class:
                now = datetime.datetime.now()
                ans.append([now.strftime('%Y-%m-%d  %H:%M:%S'),body_language_class])
            
            if body_language_class=="outofwindow":
                if count<=4:
#                     win32api.MessageBox("WARNING!: FACE NOT DETECTED")
                    count+=1
                else:
                    flag=False
#                     cap.release()
#                     win32api.MessageBox("END OF METTING")
                        
                    break

                    
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            # Get status box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            
            

            
        except Exception as e:
            print(e)
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Happy [2.06831300e-08 9.85508480e-01 1.69612517e-08 2.40230084e-08
 1.80096440e-09 1.44914568e-02]
Happy [2.06631153e-08 9.84554820e-01 7.93009795e-08 2.39997617e-08
 1.79922163e-09 1.54450547e-02]
Happy [2.06902349e-08 9.87801737e-01 4.91921012e-08 2.40312606e-08
 1.80158305e-09 1.21981673e-02]
Happy [2.09437093e-08 9.99952916e-01 4.97947496e-08 2.43256656e-08
 1.82365410e-09 4.69872984e-05]
Happy [2.06892191e-08 9.87802336e-01 4.91896860e-08 2.40300808e-08
 1.80149460e-09 1.21975684e-02]
Happy [2.06677962e-08 9.99953536e-01 4.91387450e-08 2.40051986e-08
 1.79962922e-09 4.63682862e-05]
Happy [2.06677962e-08 9.99953536e-01 4.91387450e-08 2.40051986e-08
 1.79962922e-09 4.63685667e-05]
Happy [2.00150656e-08 9.85038967e-01 2.39893079e-07 2.32470661e-08
 1.74269163e-09 1.49607476e-02]
Happy [2.02752247e-08 9.84844593e-01 2.43085794e-07 2.35492352e-08
 1.76534342e-09 1.51551181e-02]
Happy [2.09437093e-08 9.99952916e-01 4.97947496e-08 2.43256656e-08
 1.82365410e-09 4.69872984e-05]
Happy [2.0

Happy [2.11617974e-08 9.99999329e-01 1.54771929e-08 2.45789702e-08
 1.84259836e-09 6.07889533e-07]
Happy [2.12433066e-08 9.99999326e-01 1.55368066e-08 2.46736414e-08
 1.84969552e-09 6.10331929e-07]
Happy [2.11617900e-08 9.99998978e-01 3.66341729e-07 2.45789616e-08
 1.84259771e-09 6.08208304e-07]
Happy [2.09905576e-08 9.99999293e-01 5.64682668e-08 2.43803719e-08
 1.82771014e-09 6.03247951e-07]
Happy [2.09082576e-08 9.99999286e-01 6.62186684e-08 2.42847811e-08
 1.82054404e-09 6.00879077e-07]
Happy [3.24602123e-03 9.96753276e-01 7.75378851e-08 2.42059524e-08
 1.81463453e-09 5.98928616e-07]
Happy [3.23945809e-03 9.96759830e-01 8.79665331e-08 2.41570104e-08
 1.81096552e-09 5.97717641e-07]
Happy [3.13907323e-03 9.96859502e-01 8.21770854e-07 2.33705803e-08
 1.75190742e-09 5.78256015e-07]
Happy [3.13907326e-03 9.96859512e-01 8.11509550e-07 2.33705805e-08
 1.75190744e-09 5.78256021e-07]
Happy [3.13692079e-03 9.96861525e-01 9.51404271e-07 2.33545553e-08
 1.75070615e-09 5.77857990e-07]
Happy [2.0

Happy [2.05418087e-08 9.87769169e-01 1.68453608e-08 2.38588668e-08
 1.78865898e-09 1.22307675e-02]
Happy [2.05418087e-08 9.87769169e-01 1.68453608e-08 2.38588668e-08
 1.78865898e-09 1.22307675e-02]
Happy [2.05940565e-08 9.90281544e-01 1.68882067e-08 2.39195514e-08
 1.79320840e-09 9.71839314e-03]
Happy [2.05940565e-08 9.90281544e-01 1.68882067e-08 2.39195514e-08
 1.79320840e-09 9.71839314e-03]
Happy [2.05940565e-08 9.90281544e-01 1.68882067e-08 2.39195514e-08
 1.79320840e-09 9.71839314e-03]
Happy [2.05940565e-08 9.90281544e-01 1.68882067e-08 2.39195514e-08
 1.79320840e-09 9.71839314e-03]
Happy [2.05940565e-08 9.90281544e-01 1.68882067e-08 2.39195514e-08
 1.79320840e-09 9.71839314e-03]
Happy [2.04774683e-08 9.84675310e-01 1.67925983e-08 2.37841368e-08
 1.78305660e-09 1.53246271e-02]
Happy [2.05418087e-08 9.87769169e-01 1.68453608e-08 2.38588668e-08
 1.78865898e-09 1.22307675e-02]
Happy [2.05418087e-08 9.87769169e-01 1.68453608e-08 2.38588668e-08
 1.78865898e-09 1.22307675e-02]
Happy [2.0

In [10]:
file=open('body_language.txt','w')
for item in ans:
    file.write(item[0]+"-"+item[1]+"\n")
file.close()